In [1]:
# Libraries/Packages to load
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
#install.packages("themis")
install.packages("kknn")
#library(themis)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [2]:
# Reading the data from the web, adding column names
heart_data <- read_csv("https://raw.githubusercontent.com/Mr-Slope/DSCI-100_Group_Project/main/processed.cleveland.data",
                       col_names=c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))
# Preview the data set
head(heart_data)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [3]:
# Cleaning & Wrangling
heart_tidy <- heart_data |>
    filter(ca != "?", thal != "?") |>
    mutate(across(c(ca, thal), as.numeric)) |>
    mutate(across(c(cp, fbs, restecg, exang, slope, ca, thal, num), as_factor)) |>
    mutate(cp = fct_recode(cp, "angina" = "1", "abnormal" = "2", "nonang" = "3", "asymp" = "4")) |>
    mutate(fbs = fct_recode(fbs, "high" = "1", "norm" = "0")) |>
    mutate(restecg = fct_recode(restecg, "norm" = "0", "abnorm" = "1", "damage" = "2")) |>
    mutate(exang = fct_recode(exang, "TRUE" = "1", "FALSE" = "0")) |>
    mutate(slope = fct_recode(slope, "up" = "1", "flat" = "2", "down" = "3")) |>
    mutate(thal = fct_recode(thal, "norm" = "3", "fixed" = "6", "reversible" = "7")) |>
    mutate(num = fct_recode(num, "healthy" = "0", "sick" = "1", "sick" = "2", "sick" = "3", "sick" = "4")) |> # in the data files, 1,2,3,4 are all sick
    tibble() 

In [4]:
# KNN Classification model with predictions for sex, sex + MH, age + sex + mh
set.seed(29)

# Splitting the data (separate this)
heart_split <- initial_split(heart_tidy, prop = 0.75, strata = num)
heart_training <- training(heart_split)
heart_testing <- testing(heart_split)

# model 
heart_tune <- nearest_neighbor(weight="rectangular", neighbors=tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

heart_vfold <- vfold_cv(heart_training, v = 5, strata = num)
gridvals <- tibble(neighbors = seq(1,100,1))

# Thalach
heart_recipe_t <- recipe(num ~ thalach, data = heart_training) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

heart_tune_t <- workflow() |>
    add_recipe(heart_recipe_t) |>
    add_model(heart_tune) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics()

# Sex & Cholesterol
heart_recipe_sc <- recipe(num ~ sex + chol, data = heart_training) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

heart_tune_sc <- workflow() |>
    add_recipe(heart_recipe_sc) |>
    add_model(heart_tune) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics()

# Cholesterol & Thalach 
heart_recipe_ct <- recipe(num ~ chol + thalach, data = heart_training) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

heart_tune_ct <- workflow() |>
    add_recipe(heart_recipe_ct) |>
    add_model(heart_tune) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics()

# Sex & Chol & Thalach
heart_recipe_sct <- recipe(num ~ sex+ chol + thalach, data = heart_training) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

heart_tune_sct <- workflow() |>
    add_recipe(heart_recipe_sct) |>
    add_model(heart_tune) |>
    tune_grid(resamples = heart_vfold, grid = gridvals) |>
    collect_metrics()

In [10]:
set.seed(29)
# accuracies
heart_fit_t_acc <- heart_tune_t |>
    select(neighbors, mean, .metric) |>
    filter(.metric == "accuracy") |>
    arrange(-mean) |>
    slice(1)
    
heart_fit_t_acc

heart_fit_sc_acc <- heart_tune_sc |>
    select(neighbors, mean, .metric) |>
    filter(.metric == "accuracy") |>
    arrange(-mean) |>
    slice(1)
heart_fit_sc_acc

heart_fit_ct_acc <- heart_tune_ct |>
    select(neighbors, mean, .metric) |>
    filter(.metric == "accuracy") |>
    arrange(-mean) |>
    slice(1)
heart_fit_ct_acc

heart_fit_sct_acc <- heart_tune_sct |>
    select(neighbors, mean, .metric) |>
    filter(.metric == "accuracy") |>
    arrange(-mean) |>
    slice(1)
heart_fit_sct_acc

neighbors,mean,.metric
<dbl>,<dbl>,<chr>
31,0.7119192,accuracy


neighbors,mean,.metric
<dbl>,<dbl>,<chr>
57,0.6438384,accuracy


neighbors,mean,.metric
<dbl>,<dbl>,<chr>
6,0.7030303,accuracy


neighbors,mean,.metric
<dbl>,<dbl>,<chr>
35,0.7114141,accuracy


In [6]:
set.seed(29)
heart_spec_t <- nearest_neighbor(weight="rectangular", neighbors=31) |>
    set_engine("kknn") |>
    set_mode("classification")

heart_fit_t <- workflow() |>
    add_recipe(heart_recipe_t) |>
    add_model(heart_spec_t) |>
    fit(heart_training) 

heart_pred_t <- heart_fit_t |>
    predict(heart_testing) |>
    bind_cols(heart_testing) 
heart_pred_t_metrics <- heart_pred_t |>
    metrics(truth = num, estimate=.pred_class) |>
    filter(.metric == "accuracy")
heart_pred_t_metrics

heart_pred_t_conf <- heart_pred_t |>
    conf_mat(truth=num, estimate=.pred_class)
heart_pred_t_conf

# predicting with sick = true
heart_t_precision <- 23/(23+8)
heart_t_precision
heart_t_recall <- 23/(23+12)
heart_t_recall


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7333333


          Truth
Prediction healthy sick
   healthy      32   12
   sick          8   23

[1] 0.7419355

[1] 0.6571429

In [7]:
set.seed(29)
heart_spec_sc <- nearest_neighbor(weight="rectangular", neighbors=57) |>
    set_engine("kknn") |>
    set_mode("classification")

heart_fit_sc <- workflow() |>
    add_recipe(heart_recipe_sc) |>
    add_model(heart_spec_sc) |>
    fit(heart_training) 

heart_pred_sc <- heart_fit_sc |>
    predict(heart_testing) |>
    bind_cols(heart_testing) 
heart_pred_sc_metrics <- heart_pred_sc |>
    metrics(truth = num, estimate=.pred_class) |>
    filter(.metric == "accuracy")
heart_pred_sc_metrics

heart_pred_sc_conf <- heart_pred_sc |>
    conf_mat(truth=num,estimate=.pred_class)
heart_pred_sc_conf

heart_sc_precision <- 13/(13+5)
heart_sc_precision
heart_sc_recall <- 13/(13+22)
heart_sc_recall

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.64


          Truth
Prediction healthy sick
   healthy      35   22
   sick          5   13

[1] 0.7222222

[1] 0.3714286

In [8]:
set.seed(29)
heart_spec_ct <- nearest_neighbor(weight="rectangular", neighbors=6) |>
    set_engine("kknn") |>
    set_mode("classification")

heart_fit_ct <- workflow() |>
    add_recipe(heart_recipe_ct) |>
    add_model(heart_spec_ct) |>
    fit(heart_training) 

heart_pred_ct <- heart_fit_ct |>
    predict(heart_testing) |>
    bind_cols(heart_testing) 
heart_pred_ct_metrics <- heart_pred_ct |>
    metrics(truth = num, estimate=.pred_class) |>
    filter(.metric == "accuracy")
heart_pred_ct_metrics

heart_pred_ct_conf <- heart_pred_ct |>
    conf_mat(truth=num,estimate=.pred_class)
heart_pred_ct_conf

heart_ct_precision <- 22/(22+10)
heart_ct_precision
heart_ct_recall <- 22/(22+13)
heart_ct_recall

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.6933333


          Truth
Prediction healthy sick
   healthy      30   13
   sick         10   22

[1] 0.6875

[1] 0.6285714

In [11]:
set.seed(29)
heart_spec_sct <- nearest_neighbor(weight="rectangular", neighbors=69) |>
    set_engine("kknn") |>
    set_mode("classification")

heart_fit_sct <- workflow() |>
    add_recipe(heart_recipe_sct) |>
    add_model(heart_spec_sct) |>
    fit(heart_training) 

heart_pred_sct <- heart_fit_sct |>
    predict(heart_testing) |>
    bind_cols(heart_testing) 
heart_pred_sct_metrics <- heart_pred_sct |>
    metrics(truth = num, estimate=.pred_class) |>
    filter(.metric == "accuracy")
heart_pred_sct_metrics

heart_pred_sct_conf <- heart_pred_sct |>
    conf_mat(truth=num,estimate=.pred_class)
heart_pred_sct_conf

heart_sct_precision <- 24/(24+5)
heart_sct_precision
heart_sct_recall <- 24/(24+11)
heart_sct_recall

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.76


          Truth
Prediction healthy sick
   healthy      34   12
   sick          6   23

[1] 0.8275862

[1] 0.6857143

In [14]:

# # KNN Classification model with predictions for sex, sex + MH, age + sex + mh
# set.seed(29)

# # Splitting the data (separate this)
# heart_split <- initial_split(heart_tidy, prop = 0.75, strata = num)
# heart_training <- training(heart_split)
# heart_testing <- testing(heart_split)

# # model 
# heart_tune <- nearest_neighbor(weight="rectangular", neighbors=tune()) |>
#     set_engine("kknn") |>
#     set_mode("classification")

# heart_vfold <- vfold_cv(heart_training, v = 5, strata = num)
# gridvals <- tibble(neighbors = seq(1,100,1))
# # Sex & Chol & Thalach
# heart_recipe_sct <- recipe(num ~ sex+ chol + thalach, data = heart_training) |>
#     step_center(all_predictors()) |>
#     step_scale(all_predictors())

# heart_tune_sct <- workflow() |>
#     add_recipe(heart_recipe_sct) |>
#     add_model(heart_tune) |>
#     tune_grid(resamples = heart_vfold, grid = gridvals) |>
#     collect_metrics()

# heart_fit_sct_acc <- heart_tune_sct |>
#     select(neighbors, mean, .metric) |>
#     filter(.metric == "accuracy") |>
#     arrange(-mean) |>
#     slice(1)
# heart_fit_sct_acc

# # heart_spec_sct <- nearest_neighbor(weight="rectangular", neighbors=35) |>
# #     set_engine("kknn") |>
# #     set_mode("classification")

# # heart_fit_sct <- workflow() |>
# #     add_recipe(heart_recipe_sct) |>
# #     add_model(heart_spec_sct) |>
# #     fit(heart_training) 

# # heart_pred_sct <- heart_fit_sct |>
# #     predict(heart_testing) |>
# #     bind_cols(heart_testing) 
# # heart_pred_sct_metrics <- heart_pred_sct |>
# #     metrics(truth = num, estimate=.pred_class) |>
# #     filter(.metric == "accuracy")
# # heart_pred_sct_metrics

# # heart_pred_sct_conf <- heart_pred_sct |>
# #     conf_mat(truth=num,estimate=.pred_class)
# # heart_pred_sct_conf

# # heart_sct_precision <- 24/(24+5)
# # heart_sct_precision
# # heart_sct_recall <- 24/(24+11)
# # heart_sct_recall